In [2]:
import io
import itertools

import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [3]:
#loading the data
train_data = np.load(r'datasets/Trousers & Jeans - All - Train.npz')
val_data = np.load(r'datasets/Trousers & Jeans - All - Validation.npz')
test_data = np.load(r'datasets/Trousers & Jeans - All - Test.npz')

In [4]:
train_images, train_labels = train_data['images'], train_data['labels']
val_images, val_labels = val_data['images'], val_data['labels']
test_images, test_labels = test_data['images'], test_data['labels']

In [5]:
#feature scaling
train_images = train_images/255.
val_images = val_images/255.
test_images = test_images/255.

In [6]:
#constant definition
NUM_EPOCHS = 15
BATCH_SIZE = 64

In [7]:
#hyperparam definition
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([5,7]))
HP_NUM_FILTER = hp.HParam('filters_number', hp.Discrete([64,96,128]))
HP_FILTER_SIZE_2 = hp.HParam('filter_size_2', hp.Discrete([3,5]))
HP_DENSE_SIZE = hp.HParam('dense_size', hp.Discrete([512,1024]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'D:/sam/Logs/Model 4/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams= [HP_FILTER_SIZE,HP_NUM_FILTER,HP_FILTER_SIZE_2,HP_DENSE_SIZE],
        metrics = [hp.Metric(METRIC_ACCURACY,display_name='accuracy')]
    )

In [9]:
#model definition and training
def train_test_model(hparams,session_num):
    
    #model outline
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE_2],activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(4)
    ])
    
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer='adam', loss= loss_fn, metrics=['accuracy'])
    
    #defining the logging dir
    log_dir = 'D:/sam/Logs/Model 4/fit/'+'run-{}'.format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label') 
        return figure

    def plot_to_image(figure):
    
        #save the plot to a png in memory
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
    
        #closing the figure to prevent from displaying in the notebook
        plt.close(figure)
    
        buf.seek(0)
    
        image = tf.image.decode_png(buf.getvalue(), channels=4)
    
        #add the batch dimension
        image = tf.expand_dims(image, 0)
    
        return image

    #file writer for logging cofusion matrix
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    def log_confusion_matrix(epoch, logs):
    #predict values for validation set
        test_pred_r = model.predict(val_images)
        test_pred = np.argmax(test_pred_r, axis=1)
    
        cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
    
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male', 'Jeans Male', 'Trousers Female', 'Jeans Female'])
        cm_image = plot_to_image(figure)
    
        #log the confusion matrix as image summary
        with file_writer_cm.as_default():
            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            
    
    
    #defining callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= log_confusion_matrix)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode = 'auto',
        verbose=0,
        patience = 2,
        min_delta = 0,
        restore_best_weights= True
    )

    
    #training the model
    model.fit(
        train_images,
        train_labels,
        validation_data= (val_images,val_labels),
        verbose=2,
        epochs= NUM_EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks=[tensorboard_callback,cm_callback,early_stopping]
        
    )
    
    #model performance evaluation
    _, accuracy = model.evaluate(val_images,val_labels)
    
    #saving the model
    model.save(r'D:/sam/saved Models/Model 4/Run-{}'.format(session_num))
    
    return accuracy


In [11]:
#logging the result function
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [12]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for num_filter in HP_NUM_FILTER.domain.values:
        for filter_size_2 in HP_FILTER_SIZE_2.domain.values:
            for dense_size in HP_DENSE_SIZE.domain.values:
        
                hparams = {
                    HP_FILTER_SIZE : filter_size,
                    HP_NUM_FILTER : num_filter,
                    HP_FILTER_SIZE_2: filter_size_2,
                    HP_DENSE_SIZE: dense_size
                }
        
                run_name = 'run-%d' %session_num
                print('---- Starting trial : %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('D:/sam/Logs/Model 4/hparam_tuning/' +run_name, hparams, session_num)
        
                session_num += 1

---- Starting trial : run-1
{'filter_size': 5, 'filters_number': 64, 'filter_size_2': 3, 'dense_size': 512}
Epoch 1/15
63/63 - 49s - loss: 1.3557 - accuracy: 0.4720 - val_loss: 0.9992 - val_accuracy: 0.6200
Epoch 2/15
63/63 - 47s - loss: 0.8803 - accuracy: 0.6301 - val_loss: 1.0407 - val_accuracy: 0.5720
Epoch 3/15
63/63 - 45s - loss: 0.7867 - accuracy: 0.6873 - val_loss: 0.9162 - val_accuracy: 0.5920
Epoch 4/15
63/63 - 45s - loss: 0.7352 - accuracy: 0.7025 - val_loss: 0.8799 - val_accuracy: 0.6420
Epoch 5/15
63/63 - 47s - loss: 0.6705 - accuracy: 0.7295 - val_loss: 0.9459 - val_accuracy: 0.6540
Epoch 6/15
63/63 - 47s - loss: 0.6425 - accuracy: 0.7428 - val_loss: 0.7365 - val_accuracy: 0.7120
Epoch 7/15
63/63 - 46s - loss: 0.5598 - accuracy: 0.7787 - val_loss: 0.7142 - val_accuracy: 0.7320
Epoch 8/15
63/63 - 48s - loss: 0.5101 - accuracy: 0.8027 - val_loss: 0.6664 - val_accuracy: 0.7400
Epoch 9/15
63/63 - 47s - loss: 0.4758 - accuracy: 0.8129 - val_loss: 0.6848 - val_accuracy: 0.7440
E

Epoch 7/15
63/63 - 100s - loss: 0.5971 - accuracy: 0.7585 - val_loss: 0.7779 - val_accuracy: 0.6660
Epoch 8/15
63/63 - 103s - loss: 0.5591 - accuracy: 0.7742 - val_loss: 0.7996 - val_accuracy: 0.6960
Epoch 9/15
63/63 - 103s - loss: 0.5289 - accuracy: 0.7835 - val_loss: 0.7434 - val_accuracy: 0.7120
Epoch 10/15
63/63 - 100s - loss: 0.4766 - accuracy: 0.7967 - val_loss: 0.6868 - val_accuracy: 0.7500
Epoch 11/15
63/63 - 100s - loss: 0.4440 - accuracy: 0.8234 - val_loss: 0.7680 - val_accuracy: 0.7120
Epoch 12/15
63/63 - 105s - loss: 0.4012 - accuracy: 0.8427 - val_loss: 0.7263 - val_accuracy: 0.7440
16/16 [==============================] - 3s 197ms/step - loss: 0.6868 - accuracy: 0.7500
INFO:tensorflow:Assets written to: D:/sam/saved Models/Model 4/Run-7\assets
---- Starting trial : run-8
{'filter_size': 5, 'filters_number': 96, 'filter_size_2': 5, 'dense_size': 1024}
Epoch 1/15
63/63 - 116s - loss: 1.6307 - accuracy: 0.4241 - val_loss: 1.1825 - val_accuracy: 0.4520
Epoch 2/15
63/63 - 109s

Epoch 11/15
63/63 - 56s - loss: 0.4736 - accuracy: 0.8042 - val_loss: 0.7329 - val_accuracy: 0.7260
16/16 [==============================] - 2s 99ms/step - loss: 0.7157 - accuracy: 0.7080
INFO:tensorflow:Assets written to: D:/sam/saved Models/Model 4/Run-13\assets
---- Starting trial : run-14
{'filter_size': 7, 'filters_number': 64, 'filter_size_2': 3, 'dense_size': 1024}
Epoch 1/15
63/63 - 70s - loss: 1.4511 - accuracy: 0.4638 - val_loss: 1.0641 - val_accuracy: 0.5900
Epoch 2/15
63/63 - 60s - loss: 0.9726 - accuracy: 0.6031 - val_loss: 1.0019 - val_accuracy: 0.6040
Epoch 3/15
63/63 - 61s - loss: 0.8739 - accuracy: 0.6404 - val_loss: 0.9257 - val_accuracy: 0.6580
Epoch 4/15
63/63 - 59s - loss: 0.7842 - accuracy: 0.6886 - val_loss: 0.8628 - val_accuracy: 0.6420
Epoch 5/15
63/63 - 64s - loss: 0.7443 - accuracy: 0.6928 - val_loss: 0.8402 - val_accuracy: 0.6760
Epoch 6/15
63/63 - 67s - loss: 0.6870 - accuracy: 0.7195 - val_loss: 0.9164 - val_accuracy: 0.6380
Epoch 7/15
63/63 - 65s - loss: 

63/63 - 114s - loss: 1.0018 - accuracy: 0.5902 - val_loss: 1.0272 - val_accuracy: 0.5580
Epoch 3/15
63/63 - 112s - loss: 0.8898 - accuracy: 0.6361 - val_loss: 0.9441 - val_accuracy: 0.6140
Epoch 4/15
63/63 - 114s - loss: 0.8161 - accuracy: 0.6646 - val_loss: 0.9218 - val_accuracy: 0.6280
Epoch 5/15
63/63 - 113s - loss: 0.7727 - accuracy: 0.6778 - val_loss: 0.9180 - val_accuracy: 0.6220
Epoch 6/15
63/63 - 113s - loss: 0.7194 - accuracy: 0.7075 - val_loss: 0.8183 - val_accuracy: 0.6820
Epoch 7/15
63/63 - 112s - loss: 0.6605 - accuracy: 0.7333 - val_loss: 0.8427 - val_accuracy: 0.6680
Epoch 8/15
63/63 - 113s - loss: 0.6471 - accuracy: 0.7425 - val_loss: 0.8435 - val_accuracy: 0.6800
16/16 [==============================] - 3s 206ms/step - loss: 0.8183 - accuracy: 0.6820
INFO:tensorflow:Assets written to: D:/sam/saved Models/Model 4/Run-20\assets
---- Starting trial : run-21
{'filter_size': 7, 'filters_number': 128, 'filter_size_2': 3, 'dense_size': 512}
Epoch 1/15
63/63 - 108s - loss: 1.8

In [7]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"D:/sam/saved Models/Model 4/Run-3")

In [9]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)

16/16 [==============================] - 2s 103ms/step - loss: 0.7253 - accuracy: 0.7580


In [10]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.7253. Test accuracy: 75.80%


In [12]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 4/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 4/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 15704), started 0:01:25 ago. (Use '!kill 15704' to kill it.)